# 4.1 Tensorboard 可视化好帮手 2

注意: 本节内容会用到浏览器, 而且与 tensorboard 兼容的浏览器是 Google Chrome. 使用其他的浏览器不保证所有内容都能正常显示.

上一篇讲到了 如何可视化TesorBorad整个神经网络结构的过程。 其实tensorboard还可以可视化训练过程( biase变化过程) , 这节重点讲一下可视化训练过程的图标是如何做的 。请看下图, 这是如何做到的呢？


## 制作输入源
由于这节我们观察训练过程中神经网络的变化, 所以首先要添一些模拟数据. Python 的 numpy 工具包可以帮助我们制造一些模拟数据. 所以我们先导入这个工具包:

In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


然后借助 np 中的 np.linespace() 产生随机的数字, 同时为了模拟更加真实我们会添加一些噪声, 这些噪声是通过 np.random.normal() 随机产生的.

## 在 layer 中为 Weights, biases 设置变化图表
我们层中的Weights设置变化图, tensorflow中提供了tf.histogram_summary()方法,用来绘制图片, 第一个参数是图表的名称, 第二个参数是图表要记录的变量

In [2]:
def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    # add one more layer and return the output of this layer
    layer_name = 'layer%s' % n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
            tf.summary.histogram(layer_name + '/weights', Weights)# 第一个参数是图表的名称， 第二个参数是图表要记录的变量
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
            tf.summary.histogram(layer_name + '/biases', biases)# 第一个参数是图表的名称， 第二个参数是图表要记录的变量
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
        tf.summary.histogram(layer_name + '/outputs', outputs)# 第一个参数是图表的名称， 第二个参数是图表要记录的变量
    return outputs

In [3]:
# Make up some real data
x_data = np.linspace(-1, 1, 300)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise


首先从 Input 开始：
这里指定的名称将来会在可视化的图层inputs中显示出来
使用with tf.name_scope('inputs')可以将xs和ys包含进来，形成一个大的图层，图层的名字就是with tf.name_scope()方法里的参数。

In [4]:
# define placeholder for inputs to network
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

#### add hidden layer
添加n_layer参数后, 修改成为 :

In [5]:
# add hidden layer
l1 = add_layer(xs, 1, 10, n_layer=1, activation_function=tf.nn.relu)

#### add output layer
添加n_layer参数后, 修改成为 :

In [6]:
# add output layer
prediction = add_layer(l1, 10, 1, n_layer=2, activation_function=None)

## 设置loss的变化图
Loss 的变化图和之前设置的方法略有不同. loss是在tesnorBorad 的event下面的, 这是由于我们使用的是tf.scalar_summary() 方法.

观看loss的变化比较重要. 当你的loss呈下降的趋势,说明你的神经网络训练是有效果的.
修改后的代码片段如下：

In [7]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                        reduction_indices=[1]))
    tf.summary.scalar('loss', loss)

使用with tf.name_scope()再次对train_step部分进行编辑,如下：

In [8]:
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

## 给所有训练图合并
接下来， 开始合并打包。 tf.merge_all_summaries() 方法会对我们所有的 summaries 合并到一起. 因此在原有代码片段中添加：

In [9]:
sess = tf.Session()
merged = tf.summary.merge_all()

writer = tf.summary.FileWriter("logs/", sess.graph)

init = tf.global_variables_initializer()
sess.run(init)

## 训练数据
假定给出了x_data,y_data并且训练1000次.
这些仅仅可以记录很绘制出训练的图表， 但是不会记录训练的数据。 为了较为直观显示训练过程中每个参数的变化，我们每隔上50次就记录一次结果 , 同时我们也应注意, merged 也是需要run 才能发挥作用的,所以在for循环中写下

In [10]:
for i in range(1000):
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})
    if i % 50 == 0:
        result = sess.run(merged,
                          feed_dict={xs: x_data, ys: y_data})
        writer.add_summary(result, i)

## 在 tensorboard 中查看效果
direct to the local dir and run this in terminal:
$ tensorboard --logdir=D:\0ilraypan\git\tutorials\tensorflowTUT\logs